In [18]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from pandas_datareader import data as web
import datetime

In [19]:
path = os.getcwd()

In [20]:
df = pd.read_csv(path + '/combined_data.csv')

In [21]:
df = df.iloc[:10000]

In [22]:
df.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,...,DIVERTED,AIR_TIME,FLIGHTS,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_REACHED_DEST,DIV_ARR_DELAY
0,2018,1,27,6,19977,369,11697,FLL,FL,12266,...,0.0,148.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,1,27,6,19977,368,14747,SEA,WA,14771,...,0.0,98.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,1,27,6,19977,367,11278,DCA,VA,12266,...,0.0,190.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,1,27,6,19977,366,12892,LAX,CA,13930,...,0.0,212.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,1,27,6,19977,365,12451,JAX,FL,11618,...,0.0,112.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
YEAR                     10000 non-null int64
MONTH                    10000 non-null int64
DAY_OF_MONTH             10000 non-null int64
DAY_OF_WEEK              10000 non-null int64
OP_CARRIER_AIRLINE_ID    10000 non-null int64
OP_CARRIER_FL_NUM        10000 non-null int64
ORIGIN_AIRPORT_ID        10000 non-null int64
ORIGIN                   10000 non-null object
ORIGIN_STATE_ABR         10000 non-null object
DEST_AIRPORT_ID          10000 non-null int64
DEST                     10000 non-null object
DEST_STATE_ABR           10000 non-null object
CRS_DEP_TIME             10000 non-null int64
DEP_TIME                 9980 non-null float64
DEP_DELAY_NEW            9980 non-null float64
CRS_ARR_TIME             10000 non-null int64
ARR_TIME                 9978 non-null float64
ARR_DELAY_NEW            9968 non-null float64
CANCELLED                10000 non-null float64
CANCELLA

In [ ]:
#Add daily weather data to our orignal data

In [28]:
#Feature Manipulation 

#create a date column
df['date']= df.apply(lambda x:datetime.datetime.strptime("{0} {1} {2}".format(x['YEAR'],x['MONTH'], x['DAY_OF_MONTH']), "%Y %m %d"),axis=1)


#create a unique id for each flight by using carrier airline ID + flight number
df['OP_CARRIER_AIRLINE_ID'] = df['OP_CARRIER_AIRLINE_ID'].apply(str)
df['OP_CARRIER_FL_NUM'] = df['OP_CARRIER_FL_NUM'].apply(str)
df['unique_ID'] = df['OP_CARRIER_AIRLINE_ID']+df['OP_CARRIER_FL_NUM']



In [29]:
df.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_STATE_ABR,DEST_AIRPORT_ID,...,FLIGHTS,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,DIV_REACHED_DEST,DIV_ARR_DELAY,unique_ID,date
0,2018,1,27,6,19977,369,11697,FLL,FL,12266,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19977369,2018-01-27
1,2018,1,27,6,19977,368,14747,SEA,WA,14771,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19977368,2018-01-27
2,2018,1,27,6,19977,367,11278,DCA,VA,12266,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19977367,2018-01-27
3,2018,1,27,6,19977,366,12892,LAX,CA,13930,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19977366,2018-01-27
4,2018,1,27,6,19977,365,12451,JAX,FL,11618,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19977365,2018-01-27


In [ ]:
# Converting Categorial Data to dummies/feature hasher


In [32]:
#Response Variable1 - Binary - Whether the flight is delayed

#Create the binary based on whether there's delay for this flight
df['Delay']=np.where(df['DEP_DELAY_NEW']==0,0,1)

#Create Train and Test dataset - with logistic regression
features=list(df.columns)
list1 = ['DEP_DELAY_NEW','Delay','date']
for colname in list1:
    features.remove(colname)


from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.3)
x_train = train[features]
y_train = train['Delay']
x_test = test[features]
y_test = test['Delay']


from sklearn import linear_model
model = linear_model.LogisticRegression()
model.fit(x_train,y_train)
predictions = model.predict(x_test)
actuals = np.array(y_test)
logistic_acc = model.score(x_test,actuals)
print(logistic_acc)



/Users/user/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'WA'

In [ ]:
#Response Variable2 - Binary - Whether the flight is cancalled

In [ ]:
#Response Variable3 - Categorical - depends on the length of delay

In [ ]:
#Response Variable4 - 